In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import Dict, List, Tuple

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csc_matrix

/anaconda3/envs/recsys_training/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k, reciprocal_rank

In [5]:
from recsys_training.data import Dataset
from recsys_training.evaluation import retrieval_score, compute_mae, get_relevant_items, reciprocal_rank
from recsys_training.data import Dataset, genres, preprocess_users, preprocess_items, get_user_profiles

In [6]:
ml100k_ratings_filepath = '../data/raw/ml-100k/u.data'
ml100k_item_filepath = '../data/raw/ml-100k/u.item'
ml100k_user_filepath = '../data/raw/ml-100k/u.user'

In [7]:
data = Dataset(ml100k_ratings_filepath)
data.rating_split(seed=42)

In [8]:
items = pd.read_csv(ml100k_item_filepath, sep='|', header=None,
                    names=['item', 'title', 'release', 'video_release', 'imdb_url']+genres,
                    engine='python')

In [9]:
users = pd.read_csv(ml100k_user_filepath, sep='|', header=None,
                    names=['user', 'age', 'gender', 'occupation', 'zip'])

## Preprocessing

In [10]:
min_rating = 4
ratings = data.train_ratings[data.train_ratings.rating >= min_rating]

In [11]:
items = preprocess_items(items)

In [12]:
# requires feature 'release_month' in items DataFrame
profiles = get_user_profiles(ratings, items)

In [13]:
# one-hot-encode release month
release_month_df = pd.get_dummies(items['release_month'], prefix='release_month')
items = pd.concat([items, release_month_df], axis=1)
items.drop('release_month', axis=1, inplace=True)

In [14]:
items.index = items['item'].values
items.drop('item', axis=1, inplace=True)

In [15]:
users = preprocess_users(users)

In [16]:
users = users.merge(profiles, on='user', how='left')

In [17]:
users.index = users['user'].values

In [18]:
occupation_1H = pd.get_dummies(users['occupation'], prefix='occupation')
zip_1H = pd.get_dummies(users['zip'], prefix='zip')

In [19]:
users.drop(['occupation', 'zip', 'user'], axis=1, inplace=True)
users = pd.concat([users, occupation_1H, zip_1H], axis=1)

In [20]:
users.fillna(0, inplace=True)

## Model Training

In [ ]:
k = 16
N = 10
seed = 42
learning_rate = 0.05

### CF

In [ ]:
model = LightFM(loss='bpr')

### Hybrid